# Object representation

Python has two ways of representing any object as a string:

- `repr()`: Output a string representing an object as the developer may wanna see it. That's what is shown when the debugger or the console prints it. Controlled by the `__repr__()` dunder method
- `str()`: Output a  string as the user may wanna see it. That's what is shown when you call `print()` on the object. Controlled by the `__str__()` method.

There's two more methods: `__bytes__` and `__format__`. The first is how you represent your object as a sequence of bytes and the second is used by the f-strings, the builtin `format()` and the `str.format()` method. All of them call `obj.format(format_spec)`.


In [9]:
# Vector2d object is used here to demonstrate a pythonic object
import jdc
from array import array
import math

class Vector2d:
    typecode = 'd'

    def __init__(self, x, y) -> None:
        self.x = float(x)
        self.x = float(x)
    
    def __iter__(self): # Makes the object iterable, and makes the unpacking works
        return (i for i in (self.x, self.y))
    
    def __repr__(self) -> str:
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self) # The *self only works because the object is iterable
    
    def __str__(self) -> str:
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other: object) -> bool:
        return tuple(self) == tuple(other) # Works if other is Vector2d but also works for other iterables. This could lead to a bug

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    # Since we introduced a way to export a Vector2d to bytes object. We might wanna create a new object from a bytes object
    @classmethod # Modify a method, making it available to be called directly in a class
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv) # Question: this is the same as Vector2d(memv.x, memv.y)?


ModuleNotFoundError: No module named 'jdc'

## `@classmethod` versus `@staticmethod`

Those two decorators are python specific concepts.

The first allows us to define a method that acts on a class and not on an instance. The `@classmethod` changes the way a method is called. The first argument will be the class itself, instead of a instance. It's most common use is for alternative constructors, like seen above.

The second decorator `@staticmethod` changes the method to not receive any special first argument, like `self`. Essentially, a static method is just a simple function that lives in the class scope

```python
class Demo:
    @classmethod
    def klassmeth(*args):
        return args
    
    @staticmethod
    def statmeth(*args):
        return args

Demo.klassmeth() # (<class '__main__.Demo'>,) : The class itself is the first argumnt

Demo.klassmeth('spam') # (<class '__main__.Demo'>, 'spam')
Demo.statmeth() # () : empty because it's static
Demo.statmeth('spam') # ('spam')
```

The `staticmethod` decorator is not that useful as the `classmethod`.

# Formatted output

The f-strings, the builtin function `format` and the `str.format()` call the `.__format__(format_spec)` of the object.

```python
brl = 1 /4.82
brl # 0.20746887966804978
format(brl, '0.4f') # '0.2075'
'1 BRL = {rate:0.2f} USD'.format(rate=brl) # '1 BRL = 0.21 USD'
f'1 USD = {1 / brl:0.2f} BRL' # 1 USD = 4.82 BRL'
```

Hints: `int` supports `b`and `x` to see them as binary or hexadecimal, `float` supports `f` and `%` to see them as a floating point or an percentage.

Other objects can tweak how the `format_spec` is interpreted. For example:


In [3]:
from datetime import datetime
now = datetime.now()
format(now, '%H:%M:%S')

"It's now {:%I:%M %p}".format(now)

"It's now 10:32 PM"

If the class doesn't implement `__format__`, then the `str(my_object)` is called. But if a `format_spec` is passed to the function, then a Error will be thrown.

Let's implement our `__format__` for the `Vector2d` class. We will assume that the `format_spec` will format the floats that compose our object.

In [ ]:
# inside the Vector2d class

def __format__(self, fmt_spec=''):
    components = (format(c, fmt_spec) for c in self)
    return '({}, {})'.format(*components)

# See the book on more __format__ options

# Hashable Vector2d

In order to make `vector2d` hashable, which will allow it to be used in a `dict` keys or be placed in a `set`, we need to implement `__hash__()` and `__eq__()` (already implemented). Furthermore, we need to make immutable the instances of the vector, i. e., preventing someone from doing `v1.x = 7`.

In [ ]:
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y) -> None:
        self.__x = float(x) # __x indicates x is private
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    # Other methods omitted because they use self.x and self.y, that is already supported by the @property definition

    # Hash method
    def __hash__(self): # We choose to hash the tuple that composes the vector2d representation
        return hash((self.x, self.y))

# Positional pattern matching

In order to support positional pattern matching of an object, we need to implement the `__match_args__` class variable, in which, we will list the attributes of instance that will be used in the order that they will be used

In [ ]:
# Currently Vector2d supports:
def keyword_pattern_demo(v: Vector2d) -> None:
    match v:
        case Vector2d(x=0, y=0):
            print(f'{v!r} is null')
        case Vector2d(x=0):
            print(f'{v!r} is vertical')
        case Vector2d(y=0):
            print(f'{v!r} is horizontal')
        case Vector2d(x=x, y=y) if x==y:
            print(f'{v!r} is diagonal')
        case _:
            print(f'{v!r} is awesome')
        # But if we try to do:
        case Vector2d(_, 0):
            print(f'{v!r} is horizontal')

# inside class Vector2d:
    __match_args__ = ('x', 'y')

# Private (kind of) and protected attributes

In python, we cannot create a `private` member of a class like in other programming languages such as `C++` or `Java`.

When we do `self.__x` for the class `Vector2d`, the interpreter will store the variable as `_Vector2d__x` inside the instance `__dict__` in order to prevent clashes with any class that inherits from `Vector2d`. This is called name mangling. However, several people are against this idea, because it can lead to unwanted behavior. 

Another way to "protect" a variable is the convention of a single `_` before a variable name to indicate this variable is not supposed to be edited.



# Memory saving using `__slots__`

By default, a python object is stored as a dict, that consumes a big amount of memory. We can improve this by defining a class attribute called `__slots__` that will indicate which field can be stored in a array of hidden references.

In [14]:
class Pixel:
    __slots__ = ('x', 'y')

p = Pixel()
p.__dict__ # Error: no __dict__
p.x = 10 # Ok
p.color = 'red' # Not ok, because color is not defined in the __slots__. If this was a normal class, the definition of a new attribute to p would be ok

class OpenPixel(Pixel):
    pass

op = OpenPixel()
op.__dict__ # {} : Surprise, OpenPixel has a __dict__
op.x = 8
op.__dict__ # {} : But the op.x is not available in this __dict__
op.color = 'red'
op.__dict__ # {'color': 'red'} : new variables are inserted in the __dict__

# If in OpenPixel we did __slots__ = () empty tuple, a __dict__ is not created for op. 
# If in OpenPixel we did __slots__ = ('z') then a new variable would be placed in the hidden reference array

AttributeError: 'Pixel' object has no attribute 'color'

# Overloading class attributes

A class attribute is a variable that is used by all instances of a class. However, if your functions use it as `self.my_attr`, you can create a instance of your object and then modify the class attribute. This will make the object more personalized. For instance, `typecode` is a class attribute of `Vector2d` that can be modified from `d` to `f` before calling `__bytes__`.

```python
from vector2d import Vector2d

v1 = Vector2d(1.1, 2.2)
dumpd = bytes(v1) # b'd\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@'
len(dumpd) # 17

v1.typecode = 'f'
dumpf = bytes(v1) # b'f\xcd\xcc\x8c?\xcd\xcc\x0c@'
len(dumpf) # 9

Vector2d.typecode # 'd', notice the class method was not modified
```

We could do `Vector.typecode = 'f'` to modify it for any other Vector2d created, but this is not idiomatic. Most of the time, we modify the class attribute by creating a subclass. For instance:

```python
class ShortVector2d(Vector2d):
    typecode = 'f'
```